http://pbpython.com/improve-pandas-excel-output.html

In [1]:
import pandas as pd
import numpy as np
from xlsxwriter.utility import xl_rowcol_to_cell

In [2]:
df = pd.read_excel("excel-comp-datav2.xlsx")
df.head()

,account,name,street,city,state,post-code,quota,Jan,Feb,Mar
0,211829,"Kerluke, Koepp and Hilpert",34456 Sean Highway,New Jaycob,TX,28752,110000,10000,62000,35000
1,320563,Walter-Trantow,1311 Alvis Tunnel,Port Khadijah,NC,38365,150000,95000,45000,35000
2,648336,"Bashirian, Kunde and Price",62184 Schamberger Underpass Apt. 231,New Lilianland,IA,76517,300000,91000,120000,35000
3,109996,"D'Amore, Gleichner and Bode",155 Fadel Crescent Apt. 144,Hyattburgh,ME,46021,180000,45000,120000,10000
4,121213,Bauch-Goldner,7274 Marissa Common,Shanahanchester,CA,49681,300000,162000,120000,35000


In [3]:
number_rows = len(df.index)

In [4]:
# Add some summary data using the new assign functionality in pandas 0.16
df = df.assign(total=(df['Jan'] + df['Feb'] + df['Mar']))
df.head()

,account,name,street,city,state,post-code,quota,Jan,Feb,Mar,total
0,211829,"Kerluke, Koepp and Hilpert",34456 Sean Highway,New Jaycob,TX,28752,110000,10000,62000,35000,107000
1,320563,Walter-Trantow,1311 Alvis Tunnel,Port Khadijah,NC,38365,150000,95000,45000,35000,175000
2,648336,"Bashirian, Kunde and Price",62184 Schamberger Underpass Apt. 231,New Lilianland,IA,76517,300000,91000,120000,35000,246000
3,109996,"D'Amore, Gleichner and Bode",155 Fadel Crescent Apt. 144,Hyattburgh,ME,46021,180000,45000,120000,10000,175000
4,121213,Bauch-Goldner,7274 Marissa Common,Shanahanchester,CA,49681,300000,162000,120000,35000,317000


In [5]:
df = df.assign(quota_pct=(1+(df['total'] - df['quota'])/df['quota']))
df.head()

,account,name,street,city,state,post-code,quota,Jan,Feb,Mar,total,quota_pct
0,211829,"Kerluke, Koepp and Hilpert",34456 Sean Highway,New Jaycob,TX,28752,110000,10000,62000,35000,107000,0.972727
1,320563,Walter-Trantow,1311 Alvis Tunnel,Port Khadijah,NC,38365,150000,95000,45000,35000,175000,1.166667
2,648336,"Bashirian, Kunde and Price",62184 Schamberger Underpass Apt. 231,New Lilianland,IA,76517,300000,91000,120000,35000,246000,0.820000
3,109996,"D'Amore, Gleichner and Bode",155 Fadel Crescent Apt. 144,Hyattburgh,ME,46021,180000,45000,120000,10000,175000,0.972222
4,121213,Bauch-Goldner,7274 Marissa Common,Shanahanchester,CA,49681,300000,162000,120000,35000,317000,1.056667


In [6]:
writer_orig = pd.ExcelWriter('simple.xlsx', engine='xlsxwriter')
df.to_excel(writer_orig, index=False, sheet_name='report')
writer_orig.save()

In [7]:
writer = pd.ExcelWriter('fancy.xlsx', engine='xlsxwriter')
df.to_excel(writer, index=False, sheet_name='report')

In [8]:
workbook = writer.book
worksheet = writer.sheets['report']

In [9]:
worksheet.set_zoom(90)

In [10]:
# Add a number format for cells with money.
money_fmt = workbook.add_format({'num_format': '$#,##0', 'bold': True})

In [11]:
# Add a percent format with 1 decimal point
percent_fmt = workbook.add_format({'num_format': '0.0%', 'bold': True})

In [12]:
# Total formatting
total_fmt = workbook.add_format({'align': 'right', 'num_format': '$#,##0',
                                 'bold': True, 'bottom':6})

In [13]:
# Total percent format
total_percent_fmt = workbook.add_format({'align': 'right', 'num_format': '0.0%',
                                         'bold': True, 'bottom':6})

In [14]:
# Account info columns
worksheet.set_column('B:D', 20)
# State column
worksheet.set_column('E:E', 5)
# Post code
worksheet.set_column('F:F', 10)

0

In [15]:
# Monthly columns
worksheet.set_column('G:K', 12, money_fmt)
# Quota percent columns
worksheet.set_column('L:L', 12, percent_fmt)

0

In [16]:
# Add total rows
for column in range(6, 11):
    # Determine where we will place the formula
    cell_location = xl_rowcol_to_cell(number_rows+1, column)
    # Get the range to use for the sum formula
    start_range = xl_rowcol_to_cell(1, column)
    end_range = xl_rowcol_to_cell(number_rows, column)
    # Construct and write the formula
    formula = "=SUM({:s}:{:s})".format(start_range, end_range)
    worksheet.write_formula(cell_location, formula, total_fmt)

In [17]:
# Add a total label
worksheet.write_string(number_rows+1, 5, "Total",total_fmt)
percent_formula = "=1+(K{0}-G{0})/G{0}".format(number_rows+2)
worksheet.write_formula(number_rows+1, 11, percent_formula, total_percent_fmt)

0

In [19]:
# Define our range for the color formatting
color_range = "L2:L{}".format(number_rows+1)

In [20]:
# Add a format. Light red fill with dark red text.
format1 = workbook.add_format({'bg_color': '#FFC7CE',
                               'font_color': '#9C0006'})

# Add a format. Green fill with dark green text.
format2 = workbook.add_format({'bg_color': '#C6EFCE',
                               'font_color': '#006100'})

In [21]:
# Highlight the top 5 values in Green
worksheet.conditional_format(color_range, {'type': 'top',
                                           'value': '5',
                                           'format': format2})

In [22]:
# Highlight the bottom 5 values in Red
worksheet.conditional_format(color_range, {'type': 'bottom',
                                           'value': '5',
                                           'format': format1})
writer.save()